# <center> Sharing economy:</center> <center>Effects of competition on customer satisfaction on Airbnb</center>
<center>Nicolas Jow, Tiffany Ly - Télécom Paris 2021</center>

## Importation

In [1]:
import time
import numpy as np
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from bs4 import BeautifulSoup
import requests
import pandas as pd
from lxml import html
import pyreadstat
import math

## Preprocessing the original dataset

In [2]:
dtafile = 'Data_SF_2015_2019/Data_SF_2015_2019.dta'
df = pd.read_stata(dtafile)

In [3]:
df

,listing_id,reviewer_id,Review_date,comments,scrap_date,SNAP,host_name,latitude,longitude,number_of_reviews,...,availability_30,availability_60,availability_90,availability_365,first_review,last_review,price,host_has_profile_pic,host_identity_verified,require_guest_phone_verification
0,958,6413536.0,2013-06-09,Great location near Haight Street and close to...,2015-05-04,1.0,Holly,37.769310,-122.433853,32,...,0.0,0.0,8.0,246.0,NaT,NaT,170.0,,,
1,958,33750.0,2009-11-05,We highly recommend this accomodation and agre...,2015-05-04,1.0,Holly,37.769310,-122.433853,32,...,0.0,0.0,8.0,246.0,NaT,NaT,170.0,,,
2,958,1197498.0,2014-05-16,Holly's lovely apartment is exactly as adverti...,2015-05-04,1.0,Holly,37.769310,-122.433853,32,...,0.0,0.0,8.0,246.0,NaT,NaT,170.0,,,
3,958,5975179.0,2013-06-16,Truly amazing place - great location near Haig...,2015-05-04,1.0,Holly,37.769310,-122.433853,32,...,0.0,0.0,8.0,246.0,NaT,NaT,170.0,,,
4,958,78623.0,2010-03-13,"On top of all that has been said previously, I...",2015-05-04,1.0,Holly,37.769310,-122.433853,32,...,0.0,0.0,8.0,246.0,NaT,NaT,170.0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853563,35276744,NaN,NaT,,2019-07-08,47.0,Natalie,37.786621,-122.407799,0,...,2.0,2.0,2.0,271.0,NaT,NaT,28.0,t,f,f
853564,35283778,NaN,NaT,,2019-07-08,47.0,Perri,37.780621,-122.453003,0,...,0.0,0.0,2.0,2.0,NaT,NaT,120.0,t,t,f
853565,35285751,NaN,NaT,,2019-07-08,47.0,Landmark,37.754551,-122.423820,0,...,0.0,5.0,35.0,124.0,NaT,NaT,115.0,t,f,f
853566,35286441,NaN,NaT,,2019-07-08,47.0,Ko,37.758030,-122.420799,0,...,0.0,0.0,0.0,0.0,NaT,NaT,500.0,t,t,f


In [4]:
len1 = len(df)
print('We have %d rows of data' % (len1))

We have 853568 rows of data


Since we will scrape the comments given to each different guest users, we can drop the duplicate guest users, and we reset the index.

In [5]:
df = df.drop_duplicates(subset='reviewer_id', keep="first")

In [6]:
df.reset_index(drop=True, inplace=True)

In [7]:
len2 = len(df)
print('We now have %d rows of data' % (len2))

We now have 553073 rows of data


Then we can also drop all the rows where the reviewer_id is not given (N/A).

In [8]:
df = df[df['reviewer_id'].notna()]

In [9]:
df["reviewer_id"]

0           6413536.0
1             33750.0
2           1197498.0
3           5975179.0
4             78623.0
             ...     
553068     97923501.0
553069      8287696.0
553070    132464383.0
553071    116130457.0
553072    194688432.0
Name: reviewer_id, Length: 553072, dtype: float64

In [345]:
df.index[df['reviewer_id'] == 1196643].tolist()

[19254]

## Scraping code

In [359]:
# this part can be done manually

driver = webdriver.Chrome("/chromedriver")
driver.maximize_window()
url = "https://www.airbnb.fr/users/show/212954326"
website = driver.get(url)
time.sleep(5)

login = driver.find_elements_by_class_name('_m3o8wbc')
if login[3].is_displayed():
        driver.execute_script("arguments[0].click();", login[3])
        time.sleep(1)
        
driver.find_element_by_class_name("_1vf8o5y").send_keys("mail@mail.com")
driver.find_element_by_class_name("_wfo3ii").click()
sleep(1)

driver.find_element_by_class_name("_1vf8o5y").send_keys("password")
driver.find_element_by_class_name("_wfo3ii").click()



In [349]:
a= np.random.randint(20000,500000)

In [354]:
policy_date = pd.Timestamp('2017-01-01 00:00:00')

In [355]:
rows = []

for i in range(a,a+10000):
    
    review_date = df["Review_date"][i]
    if review_date < policy_date:
        continue
        
    print(str(int(df["reviewer_id"][i])))
    nb_id = str(int(df["reviewer_id"][i]))
    url = 'https://www.airbnb.fr/users/show/' + str(int(df["reviewer_id"][i]))
    page = driver.get(url)
    time.sleep(3)
    page_source = driver.page_source
    bs = BeautifulSoup(page_source, 'html.parser')
        
    # Personal information and verified identity
    
    name = bs.findAll('h1', '_14i3z6h')
    pers_info = bs.findAll('div', '_1ahen39n')
    pers_data = [info.findAll('span', '_1ax9t0a') for info in pers_info]
    member_date = bs.findAll('div', '_u5okn1')
    if len(name) <= 1: 
        continue
        
    if len(member_date) > 0:
        member_date = member_date[0].contents[0]
        
    else:
        member_date = "N/A"
        
        
    identification = bs.findAll('div', '_1i6o9hv')
    identifications = bs.findAll('div', '_l0bkzy6')
    if len(identifications)>0:
        features = [identifications[i].findAll('span', '_1ax9t0a')[0].contents[0] for i in range(int(len(identifications)/2))]
    else:
        features = "N/A"
    # Comments
        
    # Then we check if there are two tabs of comments (host/guests)
    
    if len(bs.findAll('div','_qc3ank')) == 0:
        nb_comments = 'N/A'
        if len(bs.findAll('div','_5kaapu')) > 0:
            if len(bs.findAll('div','_5kaapu')[-1].contents) >= 1: 
                nb_comments = int(bs.findAll('div','_5kaapu')[-1].contents[1][0:2])

        # We have to unroll to see if there are more than 10 comments
        
        content_coms_host = bs.findAll('div','_1v8nkzp')
        if len(content_coms_host)>0:
            nb_coms_host=nb_comments
            date_coms_host = [content_coms_host[i].contents[0].contents[0].contents[0].contents[0].contents[0] for i in range(min(10,len(content_coms_host)))]
            coms_host = [content_coms_host[i].contents[1].contents[0].contents[0]
                                  for i in range(min(10,len(content_coms_host)))]
        else:
            nb_coms_host = 0
            date_coms_host = []
            coms_host = []
            
        nb_coms_guest = 0
        coms_guest = []
        date_coms_guest = []
        
        
    else:
        coms_host = bs.findAll('div', '_qc3ank')
        coms_guest = bs.findAll('div', '_qc3ank')
        nb_coms_host = 0
        nb_coms_guest = 0
        if len(coms_host) > 0:
            nb_coms_host = coms_host[0].contents[1].contents[0]
        else:
            nb_coms_host = bs.findAll('div', '_14tkmhr')

        if len(coms_guest) > 0:
            nb_coms_guest = coms_guest[0].contents[0].contents[0]

        content_coms_guest = bs.findAll('div', attrs={'id': 'panel--user-profile-review-tabs--0'})
        date_coms_guest = []
        content_coms_host = bs.findAll('div', attrs={'id': 'panel--user-profile-review-tabs--1'})
        date_coms_host = []

        if len(content_coms_guest) > 0:
            coms_guest = [content_coms_guest[0].contents[i].findAll('span')[1].contents[0] for i in range(min(10, len(content_coms_guest[0].contents)))]
            date_coms_guest = [content_coms_guest[0].contents[i].findAll('span')[0].contents[0] for i in range(min(10, len(content_coms_guest[0].contents)))]
            #date_coms_guest = [content_coms_guest[0].contents[i].contents[0].contents[0].contents[0].contents[0].contents[0].contents[0] for i in range(len(content_coms_guest[0].contents))]
            #coms_guest = [content_coms_guest[0].contents[i].contents[0].contents[1].contents[0].contents[0] for i in range(len(content_coms_guest[0].contents))]

        if len(content_coms_host) > 0:
            coms_host = [content_coms_host[0].contents[i].findAll('span')[1].contents[0] for i in range(min(10, len(content_coms_host[0].contents)))]
            date_coms_host = [content_coms_host[0].contents[i].findAll('span')[0].contents[0] for i in range(min(10, len(content_coms_host[0].contents)))]
            #date_coms_host = [content_coms_host[0].contents[i].contents[0].contents[0].contents[0].contents[0].contents[0].contents[0] for i in range(len(content_coms_host[0].contents))]
            #coms_host = [content_coms_host[0].contents[i].contents[0].contents[1].contents[0].contents[0] for i in range(len(content_coms_host[0].contents))]

        #else:
         #   content_coms = bs.findAll('div', '_1v8nkzp')
          #  if len(bs.findAll('div', '_1v8nkzp')) > 0:
           #     content_coms_host = [com.findAll('span')[1].contents[0] for com in content_coms]
            #    dates_coms_host = bs.findAll('div', '_kb5pe')

                
    # Languages, Living Place, Job
        
    languages = []
    living_place = "N/A"
    job = "N/A"
    for data in pers_data:
        if str(data[0]).find("Langues") >= 0:
            for i in range(1, len(data[0].contents)):
                languages.append(data[0].contents[i].contents[0])

        elif str(data[0]).find("Habite") >= 0:
            living_place = data[0].contents[0]

        elif str(data[0]).find("Travail") >= 0:
            job = data[0].contents[0]
                
                
        # Adding everything
        
    rows.append({'Name': name[1].contents[0],
                'Reviewer ID': nb_id,
                'Member date': member_date,
                'Living place': living_place,
                'Languages': languages,
                'Job': job,
                'Verified features': features,
                'Nb of host coms': nb_coms_host,
                'Date of host coms': date_coms_host,
                'Host coms content': coms_host,
                'Nb of guest coms': nb_coms_guest,
                'Date of guest coms': date_coms_guest,
                'Guest coms content': coms_guest})

44869407
104331914
9884635
34063836
12163191
77590274
435058
12452884
48827995
67735340
68724120
74528
107057775
61798702
16309511
66843288
110526084
54956690
11750725
3090051
4042747
2351906
68938689
88533756
24828417
20603480
15135020
2207644
1876728
129323593
8660977
113880360
32238416
91696294
127326998
137193777
128873061
78385627
4011094
4862049
33543825
32596555
46330798
64654272
57164796
86380533
26437725
17155807
144895279
39930308
152964650
143003172
126867108
28056448
144832984
29134344
760565
137800356
44718829
171501569
141596020
156934616
34967112
96317437
38056921
3601491
53351582
37484697
64173633
1984881
173219606
174531303
12154542
4117699
36081538
49184018
16268393
128156382
31083207
75633081
5760851
61607933
161143989
32825231
172961706
138516987
204776458
83037259
207019852
57785034
124465569
176447533
211529538
215297075
76639180
204868477
70575955
14791467
220556016
35859347
177349
27669339
32369736
162485550
91197295
95830070
28252282
8635682
106762898
240319945

15754436
65850032
121579845
59676717
5855277
6076109
4654263
99297174
133455205
124882268
4516970
20121987
133820676
104843454
30855273
26633758
117527189
41011003
141716939
21335874
73435612
50164743
97994140
6018968
745689
114590783
51855040
139270744


IndexError: list index out of range

## Putting the data into a Pandas dataframe

In [356]:
guest_df = pd.DataFrame(rows)

In [357]:
guest_df

,Name,Reviewer ID,Member date,Living place,Languages,Job,Verified features,Nb of host coms,Date of host coms,Host coms content,Nb of guest coms,Date of guest coms,Guest coms content
0,"Bonjour, je m'appelle Cristian",44869407,Membre depuis 2015,"Habite à Macaé, Brésil",[],N/A,"[Identité, Adresse e-mail, Numéro de téléphone]",2,"[février 2019, janvier 2017]","[Família espetacular, grupo formidável é compr...",0,[],[]
1,"Bonjour, je m'appelle Xuan",104331914,Membre depuis 2016,Habite à Chine,[],N/A,"[Identité, Adresse e-mail, Numéro de téléphone]",1,[janvier 2017],[Xuan was very clean and quiet. Very easy to h...,0,[],[]
2,"Bonjour, je m'appelle Candy",9884635,Membre depuis 2013,Habite à Hong Kong,[],N/A,"[Identité, Adresse e-mail, Numéro de téléphone]",4,"[mars 2019, avril 2017, février 2017, novembre...",[Dear my friend\n\nOn behalf of CAO Hospitalit...,0,[],[]
3,"Bonjour, je m'appelle Faryal",34063836,Membre depuis 2015,"Habite à New York, New York","[English, Hindi]",Travail : Williams Lea Tag/WorldWriters,"[Identité, Adresse e-mail, Numéro de téléphone...",De la part des hôtes (5),"[février 2017, septembre 2016, mai 2016, juill...",[Faryal was a great guest and we'd be happy to...,De la part de voyageurs (8),"[octobre 2015, septembre 2015, septembre 2015,...",[The apartment was just as described if not be...
4,"Bonjour, je m'appelle Alexander",12163191,Membre depuis 2014,"Habite à New York, New York",[],Travail : Pillar Technologies,"[Identité, Adresse e-mail, Numéro de téléphone...",22,"[février 2021, février 2021, février 2021, jan...",[Alexander was a great guest and would host ag...,0,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,"Bonjour, je m'appelle Dii",73435612,Membre depuis 2016,"Habite à Arizona, États-Unis",[],N/A,"[Identité, Adresse e-mail, Numéro de téléphone...",5,"[octobre 2019, mars 2019, janvier 2019, novemb...",[Dii was a great guest! I hope to host her aga...,0,[],[]
914,"Bonjour, je m'appelle David",50164743,Membre depuis 2015,"Habite à Los Angeles, Californie",[],N/A,"[Identité, Adresse e-mail, Numéro de téléphone]",15,"[décembre 2019, septembre 2019, septembre 2019...","[David was a great guest!!!, David was an exce...",0,[],[]
915,"Bonjour, je m'appelle Dustin",97994140,Membre depuis 2016,"Habite à Springville, Utah",[],N/A,"[Identité, Adresse e-mail, Numéro de téléphone]",24,"[mai 2021, janvier 2021, décembre 2020, juin 2...",[Amazing guests that left our property in grea...,0,[],[]
916,"Bonjour, je m'appelle Amy",6018968,Membre depuis 2013,Habite à Singapour,[],N/A,"[Identité, Adresse e-mail, Numéro de téléphone]",20,"[mars 2021, décembre 2020, décembre 2020, nove...",[You won’t find a better Airbnb guest than Amy...,0,[],[]


In [358]:
guest_df.to_csv(r'data_scraped_more2017_1.csv', index = False)

In [288]:
guest_df["Date of host coms"][1293]

['avril 2021',
 'juillet 2018',
 'mai 2018',
 'février 2018',
 'juin 2017',
 'septembre 2016',
 'mars 2016',
 'septembre 2015',
 'août 2015',
 'juin 2015']